In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('..'))

In [2]:
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
import torch



In [3]:
import src.preprocessing as pre
import src.utils as utils

In [4]:
meta, weekly_summary, mapping_dict = pre.load_data('../data/planting_meta.json','../data/y.csv','../data/mapping_dict.json')

In [16]:
df = weekly_summary.pivot(columns='WeeksAfterTransplant', values='Kilos')

In [19]:
meta.head()

,TransplantDate,Year,WeekTransplanted,Ranch,Variety,Class,Type,Ha,WeekTransplanted_sin,WeekTransplanted_cos,ClimateSeries
2013-02-13_Felicity_ZJL_Z18_6_0.39,2013-02-13,2013,7,ZJL,Felicity,CHE,Cherry Rojo,0.3938,0.748511,0.663123,"[[-0.5605881878, -3.1935067896000002, -0.25850..."
2013-02-13_Shiren_ZJL_Z18_6_0.39,2013-02-13,2013,7,ZJL,Shiren,CHE,Cherry Rojo,0.3938,0.748511,0.663123,"[[-0.5605881878, -3.1935067896000002, -0.25850..."
2013-02-15_Amsterdam_ZJL_Z18_2_0.27,2013-02-15,2013,7,ZJL,Amsterdam,BSUF,Uva Roja,0.2700,0.748511,0.663123,"[[0.3146673828, -1.2573452111, -0.258501094600..."
2013-02-15_Felicity_ZJL_Z18_5_0.21,2013-02-15,2013,7,ZJL,Felicity,CHE,Cherry Rojo,0.2138,0.748511,0.663123,"[[0.3146673828, -1.2573452111, -0.258501094600..."
2013-02-15_Olivia_ZJL_Z18_2_0.54,2013-02-15,2013,7,ZJL,Olivia,BSUF,Uva Roja,0.5400,0.748511,0.663123,"[[0.3146673828, -1.2573452111, -0.258501094600..."


In [20]:
df = df[df.sum(axis=1) > 400]
df = df[df.nunique(axis=1) > 2]

In [22]:
smoothed = df.fillna(0).T.rolling(window=3,min_periods=1).mean()
smoothed = smoothed * df.sum(axis=1) / smoothed.sum()

In [8]:
meta = meta.loc[df.index]

In [43]:
smoothed = smoothed.T

Fit Stats

In [26]:
def logistic(t, K, r, t0):
    return K / (1 + np.exp(-r * (t - t0)))

In [45]:
results = []
for i, row in smoothed.iterrows():
    y = row.cumsum().to_numpy()
    total_kilos = y[-1]
    x = np.arange(20)
    k0 = total_kilos
    r0 = 0.01
    t0 = 10
    p0 = [k0,r0,t0]
    bounds = (
    [total_kilos * 0.95, 1e-4, 0],   # lower bounds
    [total_kilos * 1.05, 1.5, len(x)]  # upper bounds
)
    try:
        popt,pcov = curve_fit(logistic,x,y,p0=p0,bounds=bounds)
    except:
        print('didn\'t work')
        continue
    perr = np.sqrt(np.diag(pcov))           # standard deviation of parameters
    delta = 1.96 * perr                     # 95% confidence interval half-width
    results.append([popt,delta])


In [47]:
rf = pd.DataFrame([np.array(a).flatten() for a in results],columns=['K','r','t','K_err','r_err','t_err'],index=smoothed.index)

In [49]:
df1 = pd.concat([smoothed,rf],axis=1)

In [67]:
import json
mapping_dict = pre.construct_mapping_dict(meta)
with open('mapping_dict.json', 'w') as f:
    json.dump(mapping_dict, f)

In [60]:
meta = meta[meta.index.isin(df1.index)]

In [69]:
meta_encoded = meta.copy()
meta_encoded['Ranch'] = meta_encoded['Ranch'].map(mapping_dict['Ranch'])
meta_encoded['Variety'] = meta_encoded['Variety'].map(mapping_dict['Variety']) 
meta_encoded['Class'] = meta_encoded['Class'].map(mapping_dict['Class'])
meta_encoded['Type'] = meta_encoded['Type'].map(mapping_dict['Type'])
meta_encoded

,TransplantDate,Year,WeekTransplanted,Ranch,Variety,Class,Type,Ha,WeekTransplanted_sin,WeekTransplanted_cos,ClimateSeries
2013-02-13_Felicity_ZJL_Z18_6_0.39,2013-02-13,2013,7,0,0,0,0,0.3938,0.748511,0.663123,"[[-0.5605881878, -3.1935067896000002, -0.25850..."
2013-02-13_Shiren_ZJL_Z18_6_0.39,2013-02-13,2013,7,0,1,0,0,0.3938,0.748511,0.663123,"[[-0.5605881878, -3.1935067896000002, -0.25850..."
2013-02-15_Amsterdam_ZJL_Z18_2_0.27,2013-02-15,2013,7,0,2,1,1,0.2700,0.748511,0.663123,"[[0.3146673828, -1.2573452111, -0.258501094600..."
2013-02-15_Felicity_ZJL_Z18_5_0.21,2013-02-15,2013,7,0,0,0,0,0.2138,0.748511,0.663123,"[[0.3146673828, -1.2573452111, -0.258501094600..."
2013-02-15_Olivia_ZJL_Z18_2_0.54,2013-02-15,2013,7,0,3,1,1,0.5400,0.748511,0.663123,"[[0.3146673828, -1.2573452111, -0.258501094600..."
...,...,...,...,...,...,...,...,...,...,...,...
2024-07-24_Pera Amarilla Viv_VAP_V02_8_0.4,2024-07-24,2024,30,1,49,0,10,0.3971,-0.464723,-0.885456,"[[0.7705296592, 1.2392841928, 2.0075632228], [..."
2024-07-24_Top 2417_VAP_V02_8_0.18,2024-07-24,2024,30,1,52,0,8,0.1805,-0.464723,-0.885456,"[[0.7705296592, 1.2392841928, 2.0075632228], [..."
2024-07-29_King_SGB_S12_7_0.51,2024-07-29,2024,31,12,47,1,1,0.5100,-0.568065,-0.822984,"[[-0.2323673488, 0.3476308342, -0.258501094600..."
2024-07-29_King_SGB_S12_8_0.03,2024-07-29,2024,31,12,47,1,1,0.0306,-0.568065,-0.822984,"[[-0.2323673488, 0.3476308342, -0.258501094600..."


meta_encoded.to_json('planting_meta.json',orient='split')

df1.to_csv('y.csv')

In [71]:
test_meta =meta[meta['Year'] == 2024]

In [130]:
test_df = df1[df1.index.isin(test_meta.index)]

In [132]:
train_meta = meta[meta['Year'] != 2024]
train_df = df1[df1.index.isin(train_meta.index)]


Construct A Dataset Object

In [76]:
#imports
import torch
from torch.utils.data import Dataset
import torch.optim as optim
#class
class HarvestDataset(Dataset):
    def __init__(self, 
                 features,         # (N, 5)
                 ranch_ids,        # (N,)
                 class_ids,        # (N,)
                 type_ids,         # (N,)
                 variety_ids,      # (N,)
                 climate_data,     # (N, 100, 3)
                 Y_kilos = None,          # (N, 20)
                 stats = None          # (N, 6)
                ):
    

        # Convert to tensors
        self.features = torch.tensor(features, dtype=torch.float32)
        self.ranch_ids = torch.tensor(ranch_ids, dtype=torch.long)
        self.class_ids = torch.tensor(class_ids, dtype=torch.long)
        self.type_ids = torch.tensor(type_ids, dtype=torch.long)
        self.variety_ids = torch.tensor(variety_ids, dtype=torch.long)
        self.climate_data = torch.tensor(climate_data, dtype=torch.float32)
        Y_kilos = torch.tensor(Y_kilos, dtype=torch.float32)
        stats = torch.tensor(stats, dtype=torch.float32)
        self.outputs = torch.cat((Y_kilos, stats), dim=1)
        self.means = self.outputs.mean(dim=0)
        self.denom = 2*(self.outputs.max(dim=0).values - self.outputs.min(dim=0).values) + 1e-6
        self.Y = (self.outputs - self.means) / self.denom


    def __len__(self):
        return len(self.features)
    
    def get_shapes(self):
        """
        Returns a dictionary containing the shapes of all data tensors
        
        Returns
        -------
        dict
            Dictionary with tensor names as keys and their shapes as values
        """
        shapes = {
            'features': self.features.shape,
            'ranch_ids': self.ranch_ids.shape,
            'class_ids': self.class_ids.shape,
            'type_ids': self.type_ids.shape,
            'variety_ids': self.variety_ids.shape,
            'Y_kilos': self.Y.shape
        }
        return shapes
    
    def __getitem__(self, idx):
        return (
            self.features[idx],
            self.ranch_ids[idx],
            self.class_ids[idx],
            self.type_ids[idx],
            self.variety_ids[idx],
            self.climate_data[idx],
            self.Y[idx]
        )
    
    def revert(self, arr):
        return arr * self.denom + self.means

In [160]:
test_meta =meta[meta['Year'] == 2024]
test_df = df1[df1.index.isin(test_meta.index)]
train_meta = meta[meta['Year'] != 2024]
train_df = df1[df1.index.isin(train_meta.index)]

train_features = np.column_stack([
    np.array(train_meta['Ha'].values),                    # Hectares
    np.array(train_meta['WeekTransplanted_sin'].values),  # Week sine
    np.array(train_meta['WeekTransplanted_cos'].values),  # Week cosine
    np.array(train_meta['Year'].values),                  # Year
    np.ones(len(train_meta))                    # Constant feature
])
test_features = np.column_stack([
    np.array(test_meta['Ha'].values),                    # Hectares
    np.array(test_meta['WeekTransplanted_sin'].values),  # Week sine
    np.array(test_meta['WeekTransplanted_cos'].values),  # Week cosine
    np.array(test_meta['Year'].values),                  # Year
    np.ones(len(test_meta))                    # Constant feature
])
train_ranch_ids = np.array(train_meta['Ranch'].map(mapping_dict['Ranch']).values)
train_class_ids = np.array(train_meta['Class'].map(mapping_dict['Class']).values)
train_type_ids = np.array(train_meta['Type'].map(mapping_dict['Type']).values)
train_variety_ids = np.array(train_meta['Variety'].map(mapping_dict['Variety']).values)

test_ranch_ids = np.array(test_meta['Ranch'].map(mapping_dict['Ranch']).values)
test_class_ids = np.array(test_meta['Class'].map(mapping_dict['Class']).values)
test_type_ids = np.array(test_meta['Type'].map(mapping_dict['Type']).values)
test_variety_ids = np.array(test_meta['Variety'].map(mapping_dict['Variety']).values)

# Convert climate data to numpy array properly
train_climate_data = np.array([np.array(x) for x in train_meta['ClimateSeries'].values])
test_climate_data = np.array([np.array(x) for x in test_meta['ClimateSeries'].values])

train_y_kilos = train_df.values[:,:20]
test_y_kilos = test_df.values[:,:20]
train_stats = train_df.values[:,20:]
test_stats = test_df.values[:,20:]
train_dataset = HarvestDataset(train_features, train_ranch_ids, train_class_ids, train_type_ids, train_variety_ids, train_climate_data, train_y_kilos, train_stats)
test_dataset = HarvestDataset(test_features, test_ranch_ids, test_class_ids, test_type_ids, test_variety_ids, test_climate_data, test_y_kilos, test_stats)

In [167]:
train_meta

,TransplantDate,Year,WeekTransplanted,Ranch,Variety,Class,Type,Ha,WeekTransplanted_sin,WeekTransplanted_cos,ClimateSeries
2013-02-13_Felicity_ZJL_Z18_6_0.39,2013-02-13,2013,7,ZJL,Felicity,CHE,Cherry Rojo,0.3938,0.748511,0.663123,"[[-0.5605881878, -3.1935067896000002, -0.25850..."
2013-02-13_Shiren_ZJL_Z18_6_0.39,2013-02-13,2013,7,ZJL,Shiren,CHE,Cherry Rojo,0.3938,0.748511,0.663123,"[[-0.5605881878, -3.1935067896000002, -0.25850..."
2013-02-15_Amsterdam_ZJL_Z18_2_0.27,2013-02-15,2013,7,ZJL,Amsterdam,BSUF,Uva Roja,0.2700,0.748511,0.663123,"[[0.3146673828, -1.2573452111, -0.258501094600..."
2013-02-15_Felicity_ZJL_Z18_5_0.21,2013-02-15,2013,7,ZJL,Felicity,CHE,Cherry Rojo,0.2138,0.748511,0.663123,"[[0.3146673828, -1.2573452111, -0.258501094600..."
2013-02-15_Olivia_ZJL_Z18_2_0.54,2013-02-15,2013,7,ZJL,Olivia,BSUF,Uva Roja,0.5400,0.748511,0.663123,"[[0.3146673828, -1.2573452111, -0.258501094600..."
...,...,...,...,...,...,...,...,...,...,...,...
2023-09-11_Top 2323_SAB_S17_5_0.09,2023-09-11,2023,37,SAB,Top 2323,CHE,Cherry Rosa,0.0864,-0.970942,-0.239316,"[[0.369370856, 1.3921390542, 0.4553034268], [-..."
2023-09-11_Top 2417_SAB_S17_6_0.4,2023-09-11,2023,37,SAB,Top 2417,CHE,Cherry Naranja,0.4032,-0.970942,-0.239316,"[[0.369370856, 1.3921390542, 0.4553034268], [-..."
2023-09-11_Top-2245_SAB_S17_5_0.32,2023-09-11,2023,37,SAB,Top-2245,CHE,Cherry Chocolate,0.3168,-0.970942,-0.239316,"[[0.369370856, 1.3921390542, 0.4553034268], [-..."
2023-09-11_Top-2245_SAB_S17_6_0.14,2023-09-11,2023,37,SAB,Top-2245,CHE,Cherry Chocolate,0.1440,-0.970942,-0.239316,"[[0.369370856, 1.3921390542, 0.4553034268], [-..."


Pytorch Model

In [17]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

class ClimateEncoder(nn.Module):
    def __init__(self,
                 input_dim=5,
                 embedding_dim=4,
                 hidden_dim=64,
                 n_ranches=13,
                 n_classes=2,
                 n_types=14,
                 n_varieties=59,
                 climate_input_dim=3,
                 climate_hidden_dim=32):
        super().__init__()

        # Feature processing
        self.feature_encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )

        # Climate GRU
        self.climate_gru = nn.GRU(
            input_size=climate_input_dim,      # 3 features: temp_max, temp_min, precipitation
            hidden_size=climate_hidden_dim,     # You choose (maybe 32)
            batch_first=True
        )

        # Embedding dimensions
        self.ranch_dim = embedding_dim  # 12 ranches
        self.class_dim = embedding_dim  # 2 classes
        self.type_dim = embedding_dim  # 14 types
        self.variety_dim = embedding_dim  # 38 varieties

        self.ranch_emb = nn.Embedding(n_ranches, self.ranch_dim)
        self.class_emb = nn.Embedding(n_classes, self.class_dim)
        self.type_emb = nn.Embedding(n_types, self.type_dim)
        self.variety_emb = nn.Embedding(n_varieties, self.variety_dim)

        self.type_to_class = nn.Linear(self.type_dim, self.class_dim)
        self.variety_to_type = nn.Linear(self.variety_dim, self.type_dim)

        self.combined_dim = (
            hidden_dim +             # static features
            climate_hidden_dim +     # output from GRU
            self.ranch_dim + 
            self.class_dim + 
            self.type_dim + 
            self.variety_dim
        )


    def forward(self, features, ranch_id, class_id, type_id, variety_id, climate_data):
        """
        features: (batch_size, 5)
        climate_data: (batch_size, 100, 3)
        """

        # Static feature encoder
        h_features = self.feature_encoder(features)

        # Climate GRU
        batch_size = climate_data.size(0)
        h0 = torch.zeros(1, batch_size, self.climate_gru.hidden_size).to(climate_data.device)
        out, _ = self.climate_gru(climate_data, h0)  # out: (batch_size, seq_len, hidden_size)

        # Take last timestep
        climate_out = out[:, -1, :]  # (batch_size, climate_hidden_dim)

        # Embeddings
        r_emb = self.ranch_emb(ranch_id)
        c_emb = self.class_emb(class_id)
        t_emb = self.type_emb(type_id)
        v_emb = self.variety_emb(variety_id)

        # Hierarchy
        v_influence_on_type = self.variety_to_type(v_emb)
        t_emb = t_emb + v_influence_on_type

        t_influence_on_class = self.type_to_class(t_emb)
        c_emb = c_emb + t_influence_on_class

        # Combine all features
        combined = torch.cat([
            h_features,
            climate_out,
            r_emb,
            c_emb,
            t_emb,
            v_emb
        ], dim=-1)

        return combined


In [18]:
class StatsPredictor(nn.Module):
    def __init__(self,
                 encoder_dim,
                 hidden_dim = 32,   
                 output_dim = 6):
        super().__init__()

        self.output_dim = output_dim

        self.stats_predictor = nn.Sequential(
            nn.Linear(encoder_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, encoding):
        """
        encoding: (batch_size, encoder_dim)
        """
        return self.stats_predictor(encoding)

In [110]:
class HarvestModel(nn.Module):
    def __init__(self,
                 input_dim=5,
                 hidden_dim=64,
                 embedding_dim=4,
                 n_ranches=13,
                 n_classes=2,
                 n_types=14,
                 n_varieties=59,
                 climate_input_dim=3,
                 climate_hidden_dim=32,
                 output_dim=20):
        super().__init__()
        

        self.encoder = ClimateEncoder(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            embedding_dim=embedding_dim,
            n_ranches=n_ranches,
            n_classes=n_classes,
            n_types=n_types,
            n_varieties=n_varieties,
            climate_input_dim=climate_input_dim,
            climate_hidden_dim=climate_hidden_dim
        )

        self.stats_predictor = StatsPredictor(
            encoder_dim=self.encoder.combined_dim
        )
        self.t = torch.arange(output_dim, dtype=torch.float)
        
        self.final_kilos = nn.Sequential(
            nn.Linear(self.encoder.combined_dim + output_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, features, ranch_id, class_id, type_id, variety_id, climate_data):
        """
        features: (batch_size, 5)
        climate_data: (batch_size, 100, 3)
        """
        encoded = self.encoder(features, ranch_id, class_id, type_id, variety_id, climate_data)

        o2 = self.stats_predictor(encoded)
        pmf = torch.stack([self.logistic_pmf(o) for o in o2])
        together = torch.cat((encoded,pmf),dim=1)
        o1 = self.final_kilos(together)
        return torch.cat((o1,o2),dim=1)


    def logistic_pmf(self, X) -> torch.Tensor:
        # Safe clamp ranges to prevent NaNs
        K = torch.clamp(X[0], min=1e-3, max=1e4)
        r = torch.clamp(X[1], min=1e-4, max=10.0)
        t0 = torch.clamp(X[2], min=0.0, max=float(self.t[-1]))

        t = self.t
        cumulative = K / (1 + torch.exp(-r * (t - t0)))

        prepend_val = torch.zeros(1, dtype=cumulative.dtype, device=cumulative.device)
        pmf = torch.diff(cumulative, prepend=prepend_val)

        return pmf


In [164]:
len(train_dataset)

3968

In [165]:
len(train_dataset.outputs)

3150

In [161]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [162]:
lr = 1e-3
num_epochs = 35
batch_size = 32


In [163]:


# Initialize Model
model = HarvestModel()

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

model.train()

for epoch in range(num_epochs):
    total_loss = 0

    for batch in train_loader:
        features, ranch_id, class_id, type_id, variety_id, climate_data, y = batch



        # Forward pass
        outputs = model(features, ranch_id, class_id, type_id, variety_id, climate_data)
        loss = criterion(outputs, y)
        

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

IndexError: index 3267 is out of bounds for dimension 0 with size 3150

Gotta filter meta too!